In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "college-information-llm"
import pandas as pd
import csv
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [2]:
llm = ChatOpenAI(model='gpt-4o')

In [3]:
from knowledgebase import TXTKnowledgeBase

In [4]:
kb=TXTKnowledgeBase(txt_source_folder_path='lxbd')

In [5]:
#kb.initiate_documents()

In [6]:
vector=kb.return_retriever_from_persistant_vector_db()
retriever = vector.as_retriever(search_kwargs={'k':4})

In [7]:
from langchain.tools.retriever import create_retriever_tool

In [8]:
retriever_tool=create_retriever_tool(
    retriever,
    name='search_international_students_related_information',
    description='搜索并返回关于在美国留学相关的信息',
)

another tools

In [64]:
from langchain_core.tools import tool
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from colleges import CollegesData

cd=CollegesData()

college_vector=cd.return_colleges_vector_from_db()
college_retriever=college_vector.as_retriever()

@tool
def college_data(msg:str):
    """搜索美国大学数据相关的内容"""
    prompt_retriever = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="chat_history"),
        ('user',"{input}"),
        ("user", '基于聊天内容与用户输入，生成一个可以用于查询内容的学校全名，包含中文名与英文名，只回答学校全名，除此以外不回答任何内容')
    ])
    college_retriever_chain = create_history_aware_retriever(llm, college_retriever, prompt_retriever)

    class College_Info(BaseModel):
        cname:str=Field(description='学校中文全名')
        ename:str=Field(description='学校英文全名')
        postid:str=Field(description='学校的postid')
        unitid:str=Field(description='学校的unitid')
        data_type:str=Field(description='数据种类，可以是排名、录取率、录取人数、专业数、学费、犯罪率这几种中的一种或多种')
        
    parser=JsonOutputParser(pydantic_object=College_Info)
    
    prompt_document = ChatPromptTemplate.from_messages([
    ("system", "基于下面内容及用户输入，根据format_instructions输出，数据种类必须完全符合类型中的一种或几种\n\n{context}\n{format_instructions}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
    document_chain=create_stuff_documents_chain(llm,prompt_document)
    college_chain=create_retrieval_chain(college_retriever_chain,document_chain)
    response=college_chain.invoke({
        'chat_history':[{'role':'ai','content':'how can I help you?'}],
        'input':msg,
        'format_instructions':parser.get_format_instructions()
    })
    print (extract_json(response['answer']))
    return response

In [65]:
import json
import re
from typing import List
def extract_json(msg)->List[dict]:
    pattern = r"```json(.*?)```"
    matches = re.findall(pattern, msg, re.DOTALL)
    try:
        return [json.loads(match.strip()) for match in matches]
    except Exception:
        raise ValueError(f"Failed to parse: {message}")

In [66]:
tools=[retriever_tool,college_data]

In [67]:
#from langchain import hub

#prompt = hub.pull("hwchase17/openai-tools-agent")
#prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "不要改变输出内容格式"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [86]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, return_intermediate_steps=True)

In [87]:
result = agent_executor.invoke({"input": "普林斯顿大学留学"})



> Entering new AgentExecutor chain...

Invoking: `search_international_students_related_information` with `{'query': '普林斯顿大学留学'}`


【23年4月24日更新】补充申请到“野鸡大学”的不良后果 。

疫情过后，随着留学热潮的回温，越来越多的中国学生选择留学美国。在选择申请美国大学时，成绩优异的学生自然会选择美国大学排名靠前的名校，但是对于成绩一般或者偏差较大的学生来说，申请排名靠后的大学则是不得已而为之的选择。然而，一些大学虽然排名较低，但是教学质量并不差，而有一些大学却因为教学质量太差或行挂靠国际学生身份之实而被人们称为“野鸡大学”，对于这些大学，学生申请的风险极大。

首先，申请到“野鸡大学”读书的学生很难享受到美国高质量的教育。一些“野鸡大学”教学水平不高，甚至有些教授可能并不具备相关学科的专业背景，这样的大学难以为学生提供高质量的教育和良好的学术环境。另外，这些大学的课程设置可能也比较单一，缺乏多元化的学科和课程选择，很难满足学生的个性化需求。

其次，申请签证时被拒的风险也会大大增加。由于“野鸡大学”声誉不佳，签证官会对这样的申请人产生怀疑，认为他们可能是通过这种方式来绕过留学签证的限制，或者是想通过读一些质量低下的课程来获得美国的身份。因此，在申请签证时，如果选择了“野鸡大学”，那么签证被拒的风险会大大增加。

最后，申请到“野鸡大学”读书的学生学历也很可能遭到企业的质疑。在国内求职时，一些公司会对学历和学校的认可度进行严格的审核，如果申请人的学历来自于一所名不见经传的大学，那么很可能会被视为不符合公司的招聘标准。因此，申请到“野鸡大学”读书的学生不仅难以获得高质量的教育，还可能影响其未来的就业前景。

那么，怎样才能避免申请到“野鸡大学”呢？最好的方式就是选择受到美国教育部认可的大学。根据美国宪法规定，美国各院校是否被认可主要取决于其是否通过了政府承认的认证机构。目前，美国已有3600多所被认可的院校，其质量认证一般由专业认证机构负责。经过美国教育部认可的专业认证机构主要有六大区域认证机构和近百个专业认证机构。联邦政府通过认证机构间接起到对各院校的管理和规范作用。通过区域性认证协会或其他专业认证机构的认证，各院校可实现学分

In [104]:
result

{'input': '普林斯顿大学留学',
 'output': '### 普林斯顿大学留学信息\n\n#### 学校简介\n普林斯顿大学（Princeton University）成立于1746年，是一所位于美国新泽西州普林斯顿市的私立研究型大学。作为常春藤联盟的一员，普林斯顿大学在全球享有极高的声誉，其学术水平和科研实力处于世界领先地位。\n\n#### 申请要求\n1. **学术要求**：\n    - 高中毕业或同等学历；\n    - 优秀的高中成绩，特别是数学和科学课程；\n    - 需要提供SAT或ACT成绩（普林斯顿大学对于国际学生的标化考试要求可能有所不同，请访问学校官网获取最新信息）。\n\n2. **英语语言能力**：\n    - 非英语母语的申请者需要提供托福（TOEFL）或雅思（IELTS）成绩。\n    - 普林斯顿大学对于托福成绩的要求一般为100分以上，雅思为7.0分以上。\n\n3. **推荐信**：\n    - 需要提交两封教师推荐信，最好来自于数学或科学教师。\n\n4. **课外活动及领导力**：\n    - 普林斯顿大学非常重视学生的课外活动经历和领导力表现，因此申请者需展示自己在这些方面的成就。\n\n#### 申请流程\n1. **在线申请**：\n    - 通过普林斯顿大学的官网或通用申请系统（Common Application）提交申请。\n\n2. **提交申请材料**：\n    - 高中成绩单；\n    - SAT/ACT成绩；\n    - 托福/雅思成绩（如适用）；\n    - 推荐信；\n    - 个人陈述和短文。\n\n3. **面试**：\n    - 普林斯顿大学可能会要求申请者进行面试，面试通常由校友或招生委员会成员进行。\n\n#### 学费及奖学金\n- 普林斯顿大学的学费较高，但学校提供多种奖学金和助学金计划，国际学生同样有资格申请。\n- 学校根据学生的家庭经济情况提供资助，确保每一位被录取的学生都能负担得起学费。\n\n### 学校数据\n```json\n{\n  "cname": "普林斯顿大学",\n  "ename": "Princeton University",\n  "postid": "8413",\n  "unitid": "186131",\n  

ToolAgentAction(tool='college_data', tool_input={'msg': '普林斯顿大学数据'}, log="\nInvoking: `college_data` with `{'msg': '普林斯顿大学数据'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_7vbymx0DJ53PVWOzfaz65HF5', 'function': {'arguments': '{"query": "普林斯顿大学留学"}', 'name': 'search_international_students_related_information'}, 'type': 'function'}, {'index': 1, 'id': 'call_G5Mu4wNQ9067UaGWuQqrRmMF', 'function': {'arguments': '{"msg": "普林斯顿大学数据"}', 'name': 'college_data'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-7a541859-af2e-4809-9c22-e82cd5ac7c2f', tool_calls=[{'name': 'search_international_students_related_information', 'args': {'query': '普林斯顿大学留学'}, 'id': 'call_7vbymx0DJ53PVWOzfaz65HF5'}, {'name': 'college_data', 'args': {'msg': '普林斯顿大学数据'}, 'id': 'call_G5Mu4wNQ9067UaGWuQqrRmMF'}], tool_call_chunks=[{'name': 'search_international_students_related_information', 'args': '{"query": "普林斯顿大学留学"}', 'id'

In [2]:
colleges=pd.read_csv('.\\colleges\\colleges.csv')

In [15]:
embedding=OpenAIEmbeddings(model='text-embedding-3-small')

In [35]:
row_txt_format='中文名：{c}，英文名：{e}，类型：{t}，postid：{p}，unitid：{u}，所在州：{s}'
college_types={1:'综合大学',2:'文理学院',3:'社区大学'}
txts=[]
for index,row in colleges.iterrows():
    row_txt=row_txt_format.format(c=row['cname'],e=row['name'],t=college_types[row['type']],p=row['postid'],u=row['unitid'],s=row['state'])
    txts.append(row_txt)

In [36]:
vectors=FAISS.from_texts(txts,embedding)

In [41]:
vectors.save_local(folder_path='vector',index_name='colleges-data-vector')

In [37]:
retriever=vectors.as_retriever()

In [40]:
retriever.invoke('princeton')

[Document(page_content='中文名：普林斯顿大学，英文名：Princeton University，类型：综合大学，postid：8413，unitid：186131，所在州：NJ'),
 Document(page_content='中文名：普林西庇亚学院，英文名：Principia College，类型：文理学院，postid：56413，unitid：148016，所在州：IL'),
 Document(page_content='中文名：迪堡大学，英文名：DePauw University，类型：文理学院，postid：37722，unitid：150400，所在州：IN'),
 Document(page_content='中文名：布朗大学，英文名：Brown University，类型：综合大学，postid：9427，unitid：217156，所在州：RI')]